<a href="https://colab.research.google.com/github/Daz-Riza-Seriog/Tensorflow_ML/blob/main/1-Introduction%20to%20Tensorflow/3-Week%203/3_Callbacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MODEL REGULARISATION**

Import Libraries

In [1]:
import tensorflow as tf
print(tf.__version__)

2.9.2


Load the Data - Diabetes

In [2]:
from sklearn.datasets import load_diabetes

diabetes_dataset = load_diabetes()
print(diabetes_dataset["DESCR"])

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

look the Keys of the dataset

In [3]:
print(diabetes_dataset.keys())

dict_keys(['data', 'target', 'frame', 'DESCR', 'feature_names', 'data_filename', 'target_filename', 'data_module'])


Now we will load the data and the target

In [4]:
data = diabetes_dataset["data"]
target = diabetes_dataset["target"]

Look up the target array and see if is neccesarry normalize

In [5]:
print(target)

[151.  75. 141. 206. 135.  97. 138.  63. 110. 310. 101.  69. 179. 185.
 118. 171. 166. 144.  97. 168.  68.  49.  68. 245. 184. 202. 137.  85.
 131. 283. 129.  59. 341.  87.  65. 102. 265. 276. 252.  90. 100.  55.
  61.  92. 259.  53. 190. 142.  75. 142. 155. 225.  59. 104. 182. 128.
  52.  37. 170. 170.  61. 144.  52. 128.  71. 163. 150.  97. 160. 178.
  48. 270. 202. 111.  85.  42. 170. 200. 252. 113. 143.  51.  52. 210.
  65. 141.  55. 134.  42. 111.  98. 164.  48.  96.  90. 162. 150. 279.
  92.  83. 128. 102. 302. 198.  95.  53. 134. 144. 232.  81. 104.  59.
 246. 297. 258. 229. 275. 281. 179. 200. 200. 173. 180.  84. 121. 161.
  99. 109. 115. 268. 274. 158. 107.  83. 103. 272.  85. 280. 336. 281.
 118. 317. 235.  60. 174. 259. 178. 128.  96. 126. 288.  88. 292.  71.
 197. 186.  25.  84.  96. 195.  53. 217. 172. 131. 214.  59.  70. 220.
 268. 152.  47.  74. 295. 101. 151. 127. 237. 225.  81. 151. 107.  64.
 138. 185. 265. 101. 137. 143. 141.  79. 292. 178.  91. 116.  86. 122.
  72. 

Normalize the target

In [6]:
target = (target-target.mean())/target.std()
print(target)

[-1.47194752e-02 -1.00165882e+00 -1.44579915e-01  6.99512942e-01
 -2.22496178e-01 -7.15965848e-01 -1.83538046e-01 -1.15749134e+00
 -5.47147277e-01  2.05006151e+00 -6.64021672e-01 -1.07957508e+00
  3.48889755e-01  4.26806019e-01 -4.43258925e-01  2.45001404e-01
  1.80071184e-01 -1.05621783e-01 -7.15965848e-01  2.06043272e-01
 -1.09256112e+00 -1.33929596e+00 -1.09256112e+00  1.20596866e+00
  4.13819975e-01  6.47568766e-01 -1.96524090e-01 -8.71798376e-01
 -2.74440354e-01  1.69943833e+00 -3.00412442e-01 -1.20943552e+00
  2.45262887e+00 -8.45826288e-01 -1.13151925e+00 -6.51035629e-01
  1.46568953e+00  1.60853602e+00  1.29687096e+00 -8.06868156e-01
 -6.77007716e-01 -1.26137969e+00 -1.18346343e+00 -7.80896068e-01
  1.38777327e+00 -1.28735178e+00  4.91736239e-01 -1.31593871e-01
 -1.00165882e+00 -1.31593871e-01  3.72247006e-02  9.46247777e-01
 -1.20943552e+00 -6.25063541e-01  3.87847887e-01 -3.13398486e-01
 -1.30033783e+00 -1.49512849e+00  2.32015360e-01  2.32015360e-01
 -1.18346343e+00 -1.05621

Split train and test sets using sklearn package

In [7]:
from sklearn.model_selection import train_test_split

train_data, test_data,train_targets, test_targets = train_test_split(data, target,test_size=0.1)

# look the shape of the train and test
print(train_data.shape)
print(test_data.shape)
print(train_targets.shape)
print(test_targets.shape)

(397, 10)
(45, 10)
(397,)
(45,)


**INTRODUCTION TO CALLBACKS**

Create a regularisation with weight decay and Dropout

In [11]:
def get_regularised_model(wd,rate):
  Sequential = tf.keras.models.Sequential
  Dense = tf.keras.layers.Dense
  Dropout = tf.keras.layers.Dropout
  regularizers = tf.keras.regularizers
  model = Sequential([
      Dense(128, kernel_regularizer=regularizers.l2(wd), activation="relu", input_shape=(train_data.shape[1],)),
      Dropout(rate),
      Dense(128, kernel_regularizer=regularizers.l2(wd),activation="relu"),
      Dropout(rate),
      Dense(128, kernel_regularizer=regularizers.l2(wd),activation="relu"),
      Dropout(rate),
      Dense(128, kernel_regularizer=regularizers.l2(wd),activation="relu"),
      Dropout(rate),
      Dense(128, kernel_regularizer=regularizers.l2(wd),activation="relu"),
      Dropout(rate),
      Dense(128, kernel_regularizer=regularizers.l2(wd),activation="relu"),
      Dropout(rate),
      Dense(1)
  ])

  return model

Example training Callback

In [10]:
Callback = tf.keras.callbacks.Callback

class TrainingCallback(Callback):

  def on_train_begin(self, logs=None):
    print("Starting training....")
  
  def on_epoch_begin(self, epoch, logs=None):
    print(f"Starting epoch {epoch}")

  def on_train_batch_begin(self, batch, logs=None):
    print(f"Training: Starting batch {batch}")

  def on_train_batch_end(self, batch, logs=None):
    print(f"Training: Finished batch {batch}")

  def on_epoch_end(self, epoch, logs=None):
    print(f"Finished epoch {epoch}")

  def on_train_end(self, logs=None):
    print("Finished training!")


You could use to test instead of train

In [16]:
Callback = tf.keras.callbacks.Callback

class TestingCallback(Callback):

  def on_test_begin(self, logs=None):
    print("Starting testing....")
  
  def on_test_batch_begin(self, batch, logs=None):
    print(f"Testing: Starting batch {batch}")

  def on_test_batch_end(self, batch, logs=None):
    print(f"Testing: Finished batch {batch}")

  def on_test_end(self, logs=None):
    print("Finished testing!")

You could create a callback in Predict instead of the above

In [18]:
Callback = tf.keras.callbacks.Callback

class PredictionCallback(Callback):

  def on_predict_begin(self, logs=None):
    print("Starting prediction....")
  
  def on_predict_batch_begin(self, batch, logs=None):
    print(f"Prediction: Starting batch {batch}")

  def on_predic_batch_end(self, batch, logs=None):
    print(f"Prediction: Finished batch {batch}")

  def on_predict_end(self, logs=None):
    print("Finished prediction!")

Rebuild the model with the dw decay weight and dropout rate rate

In [12]:
model = get_regularised_model(1e-5,0.3)

Compile the model

In [14]:
model.compile(optimizer="adam", loss="mse")

Train the model with the Callback

In [15]:
model.fit(train_data,train_targets, epochs=3, batch_size=128, callbacks=[TrainingCallback()])

Starting training....
Starting epoch 0
Epoch 1/3
Training: Starting batch 0
Training: Finished batch 0
1/4 [======>.......................] - ETA: 3s - loss: 0.9931Training: Starting batch 1
Training: Finished batch 1
Training: Starting batch 2
Training: Finished batch 2
Training: Starting batch 3
Training: Finished batch 3
Finished epoch 0
4/4 [==============================] - 1s 9ms/step - loss: 0.9861
Starting epoch 1
Epoch 2/3
Training: Starting batch 0
Training: Finished batch 0
1/4 [======>.......................] - ETA: 0s - loss: 0.8475Training: Starting batch 1
Training: Finished batch 1
Training: Starting batch 2
Training: Finished batch 2
Training: Starting batch 3
Training: Finished batch 3
Finished epoch 1
4/4 [==============================] - 0s 9ms/step - loss: 0.9782
Starting epoch 2
Epoch 3/3
Training: Starting batch 0
Training: Finished batch 0
1/4 [======>.......................] - ETA: 0s - loss: 0.8946Training: Starting batch 1
Training: Finished batch 1
Training

You can evaluate with a Callbacks

In [17]:
model.evaluate(test_data,test_targets,callbacks=[TestingCallback()])

Starting testing....
Testing: Starting batch 0
Testing: Finished batch 0
1/2 [==============>...............] - ETA: 0s - loss: 1.2027Testing: Starting batch 1
Testing: Finished batch 1
Finished testing!
2/2 [==============================] - 0s 9ms/step - loss: 1.1073


1.1072698831558228

Make prediction with the Callback

In [19]:
model.predict(test_data, callbacks=[PredictionCallback()])

Starting prediction....
Prediction: Starting batch 0
1/2 [==============>...............] - ETA: 0sPrediction: Starting batch 1
Finished prediction!
2/2 [==============================] - 0s 8ms/step


array([[ 0.01745544],
       [-0.03035781],
       [ 0.14667411],
       [ 0.02494779],
       [-0.02253216],
       [ 0.04488185],
       [ 0.068035  ],
       [ 0.03381426],
       [ 0.03435029],
       [ 0.00199743],
       [-0.01410899],
       [ 0.04740192],
       [ 0.07686902],
       [-0.0303348 ],
       [ 0.08232178],
       [ 0.04002849],
       [ 0.04047794],
       [-0.03109516],
       [ 0.11178611],
       [-0.02661839],
       [ 0.07919888],
       [ 0.05310574],
       [-0.02528402],
       [ 0.07543404],
       [-0.02382502],
       [ 0.09921256],
       [ 0.03271949],
       [-0.00736413],
       [-0.01602827],
       [-0.00926644],
       [-0.02512121],
       [ 0.07183389],
       [ 0.03164518],
       [-0.00989757],
       [ 0.01518012],
       [ 0.06412941],
       [-0.02040743],
       [ 0.02036356],
       [ 0.11840141],
       [ 0.10862108],
       [ 0.01871417],
       [-0.01183578],
       [ 0.10702024],
       [ 0.07645875],
       [ 0.07024168]], dtype=flo

PLOT 

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [9]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("Loss vs. Epochs")
plt.ylabel("Loss")
plt.xlabel("Epochs")
plt.legend(["Training","Validation"],loc="best")
plt.show()

NameError: ignored

**Analysis**

We note that the overfitting is not correct yet, but it´s less than the no-regularised model.